In [3]:
#Imported Libraries
from selenium import webdriver
import time
import pandas as pd
import os 
import numpy as np

In [4]:
#Import Packages From Selenium
from selenium.webdriver.support.select import Select 
from selenium.webdriver.support.ui import WebDriverWait    
from selenium.webdriver.common.by import By    
from selenium.webdriver.support import expected_conditions as EC  
from selenium.webdriver.common.keys import Keys

## Opens the chrome and take login page of Linkedin

In [55]:
driver = webdriver.Chrome(executable_path=r'F:\\chromedriver.exe')  #I can control chrome with driver variable;
driver.get("https://www.linkedin.com/")                             # Gets to linkedin home page;
driver.maximize_window()                                            # Maximises the present browser or page window;

C:\Users\bjami\AppData\Local\Temp\ipykernel_10580\643997193.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r'F:\\chromedriver.exe')  #I can control chrome with driver variable;


# Automatic Login- Linkedin 

In [56]:
username = driver.find_element(By.ID, "session_key") 
password = driver.find_element(By.ID, "session_password")
username.send_keys("amithbj08@gmail.com")                                   # Founder the username box as above and sented values to that particular box;
password.send_keys("Amith@1234")
driver.find_element(By.CLASS_NAME,"sign-in-form__submit-button").click()    # Finding submit button and clicking it;

Direct link for job search in linkedin:

In [38]:
driver.get("https://www.linkedin.com/jobs/search/?currentJobId=3399692655&geoId=102713980&keywords=jobs&location=India&refresh=true")
# Getting straight to the jobs page ;

Function to find all company and their linkedin page links for getting number of followers:

In [57]:
def job_exist_function(): # this function gets all company names;
    company_names= []
    n=26
    try:
        for i in range(n):
            company=driver.find_elements(By.CLASS_NAME, 'job-card-container__company-name')[i].text     #Company name;
            company_names.append(company)
    except:
        return company_names

Function to scroll down the page till end:

In [58]:
def scroll_till_down(): # This function will scroll down and looks for all the jobs in one page;
    Assumed_company_num=26
    try:
        for num in range(Assumed_company_num):
            search=driver.find_elements(By.CLASS_NAME, 'job-card-container__company-name')[num] 
            driver.execute_script('arguments[0].scrollIntoView({behavior: "auto", block: "start", inline: "end"});', search)   
            
    except:
        pass

Function to move through all pages till p-1 pages:

In [59]:
def Checks_if_page_exist_and_clicks(p):
    s=driver.find_elements(By.CLASS_NAME,'artdeco-pagination__indicator--number') # List of pages buttons;
    for num in range(len(s)):   # Taking the buttons one at a time;
        if s[num].text==str(p): # 
            print(f"Got page {s[num].text}")
            time.sleep(1)
            s[num].click()
            break
        if num==9: # If at all the loop reach last one or 9th one press'...', but in one case it happens;
            print('Was inside')
            s[8].click() # Because 8th one is '...';
            s=driver.find_elements(By.CLASS_NAME,'artdeco-pagination__indicator--number') # List of pages buttons;
            for num in range(len(s)):
                if s[num].text==str(p):
                    print(f"Got page {s[num].text}")
                    s[num].click()
                    break

Function to extract information about the job profile in a particular linkedin:

In [60]:
def scrape_job_data():     # Function two
    job_name = driver.find_element(By.CLASS_NAME,"jobs-unified-top-card__job-title").text
    company_location = driver.find_elements(By.CLASS_NAME,"jobs-unified-top-card__bullet")[0].text
    job_type_and_level = driver.find_elements(By.CLASS_NAME, "jobs-unified-top-card__job-insight")[0].text
    employees_and_industry = driver.find_elements(By.CLASS_NAME, "jobs-unified-top-card__job-insight")[1].text
    company_name = driver.find_element(By.CSS_SELECTOR,'.ember-view.t-black.t-normal').text
    link = driver.find_element(By.CSS_SELECTOR,'.ember-view.t-black.t-normal').get_attribute('href')
    job_description = driver.find_element(By.CLASS_NAME, "jobs-description-content__text--stretch").text
    try:
        job_applicants = driver.find_element(By.CLASS_NAME, "jobs-unified-top-card__applicant-count").text
    except:
        job_applicants = "Nil"
    return [company_name, job_name, company_location, job_applicants, job_type_and_level, employees_and_industry, link, job_description]


Function to get number of followers from each companie's Linkedin page:

In [61]:
def scrape_company_followers():     # There is a chance of error occuring in followers but if error occurs then followers will be empty
    following = ""
    following = driver.find_elements(By.CLASS_NAME, "org-top-card-summary-info-list__info-item")[-1].text
    Industry = driver.find_elements(By.CLASS_NAME, "org-top-card-summary-info-list__info-item")[0].text
    return following,Industry

# Extracting all the information about the job-->

In [45]:
page=31
df = pd.DataFrame(columns = ["Company","Job_name", "Location","Applicants", "Job_level", "Employees","Link","Job_description"])
company_list=[]
company_links=[]

for p in range(1,page):
        Checks_if_page_exist_and_clicks(p)
        scroll_till_down()
        names=job_exist_function()
        total_job_num=len(names)        # Total jobs in a particular page;

        for num in range(total_job_num):
                driver.find_elements(By.CLASS_NAME,'job-card-list__title')[num].click() # Clicks the particular job profile;
                try:
                        job_details = scrape_job_data() # Some details about job;
                except:
                        time.sleep(1.5)
                        try:
                                job_details = scrape_job_data()
                        except:
                                continue
                print(f"Success got job: {num} of page: {p}")
                df.loc[len(df)] = job_details

Got page 1
Success got job: 0 of page: 1
Success got job: 1 of page: 1
Success got job: 2 of page: 1
Success got job: 3 of page: 1
Success got job: 4 of page: 1
Success got job: 5 of page: 1
Success got job: 6 of page: 1
Success got job: 7 of page: 1
Success got job: 8 of page: 1
Got page 2
Success got job: 0 of page: 2
Success got job: 1 of page: 2
Success got job: 2 of page: 2
Success got job: 3 of page: 2
Success got job: 4 of page: 2
Success got job: 6 of page: 2
Success got job: 7 of page: 2
Success got job: 8 of page: 2
Success got job: 9 of page: 2
Success got job: 10 of page: 2
Success got job: 11 of page: 2
Success got job: 12 of page: 2
Success got job: 13 of page: 2
Success got job: 14 of page: 2
Success got job: 15 of page: 2
Success got job: 16 of page: 2
Success got job: 18 of page: 2
Success got job: 19 of page: 2
Success got job: 20 of page: 2
Success got job: 22 of page: 2
Success got job: 23 of page: 2
Got page 3
Success got job: 0 of page: 3
Success got job: 1 of pag

Saving DataFrame

In [67]:
df.to_csv('saved_for_safety.csv')

From above DF taking Company names and Company Linkedin page links:

In [62]:
company_names = df.Company.values.tolist()
company_links = df.Link.values.tolist()
company_df = pd.DataFrame(data={'Company':company_names, 'links':company_links}) 

company_df=company_df.drop_duplicates()                                         # Removed duplicates
company_df.reset_index(drop=True,inplace=True)
company_df.dropna(axis=0, inplace=True)

Extracting number of Followers & Industry of a particular company:

In [68]:
followers_list = []
industry_list = []
for num, link in enumerate(company_df.links.values):
    driver.get(link)
    try:
        followers, industry = scrape_company_followers()
    except:
        time.sleep(1.5)
        try:
            followers, industry = scrape_company_followers()
        except:
            followers, industry = np.nan, np.nan
    followers_list.append(followers)
    industry_list.append(industry)

Converting to a DF:

In [69]:
company_df['Followers'] = followers_list
company_df['Industry'] = industry_list
company_df = company_df[['Company','Followers','Industry']]
company_df

,Company,Followers,Industry
0,BYJU'S,"1,686,546 followers",E-Learning Providers
1,Walmart Global Tech India,"836,404 followers",IT Services and IT Consulting
2,Target,"1,619,531 followers",Retail
3,Appen,"865,805 followers",IT Services and IT Consulting
4,WM,"233,509 followers",Environmental Services
...,...,...,...
257,OfficeDay Innovation,"1,197 followers",Human Resources Services
258,VEGA Intellisoft Pvt Ltd,"9,775 followers",IT Services and IT Consulting
259,PepsiCo,"6,927,538 followers",Food and Beverage Services
260,Picostone,"1,780 followers",Computers and Electronics Manufacturing


# Joined Followers df and Job_details df

In [70]:
new_df=df.join(company_df.set_index(keys='Company'), on='Company')
new_df.isnull().sum()
new_df.dropna(axis=0,inplace=True)
new_df.reset_index(inplace=True, drop=True)
new_df

,Company,Job_name,Location,Applicants,Job_level,Employees,Link,Job_description,Followers,Industry
0,BYJU'S,Business Development Associate,"Bengaluru, Karnataka, India",11 applicants,Full-time,"10,001+ employees",https://www.linkedin.com/company/byjus/life/,Expectation from you: You will begin as an ind...,"1,686,546 followers",E-Learning Providers
1,Walmart Global Tech India,PROCESS SPECIALIST,"Bengaluru, Karnataka, India",46 applicants,Full-time · Entry level,"5,001-10,000 employees · IT Services and IT Co...",https://www.linkedin.com/company/walmartglobal...,Position Summary...\n\nDemonstrates up-to-date...,"836,404 followers",IT Services and IT Consulting
2,Target,Data Sciences Intern V,"Bengaluru, Karnataka, India",73 applicants,Full-time · Internship,"10,001+ employees · Retail",https://www.linkedin.com/company/target/life/,Description\n\nPresenting a program that provi...,"1,619,531 followers",Retail
3,BYJU'S,Business Development Associate,India,3 applicants,Full-time,"10,001+ employees",https://www.linkedin.com/company/byjus/life/,We take pride in letting you know that you are...,"1,686,546 followers",E-Learning Providers
4,Appen,Real-Life Video Collector,"Maharashtra, India",9 applicants,Part-time · Entry level,"501-1,000 employees · IT Services and IT Consu...",https://www.linkedin.com/company/appen/life/,Help us work with a leading technology provide...,"865,805 followers",IT Services and IT Consulting
...,...,...,...,...,...,...,...,...,...,...
634,Mondelēz International,BDE-Biscuits-Cochin,India,32 applicants,Full-time,"10,001+ employees · Food and Beverage Manufact...",https://www.linkedin.com/company/mondelezinter...,Job Description\n\nAre You Ready to Make It Ha...,"2,248,950 followers",Food and Beverage Manufacturing
635,State Street,Core Operations - Manager,"Bengaluru, Karnataka, India",Nil,Full-time,"10,001+ employees · Financial Services",https://www.linkedin.com/company/state-street/...,Performs daily maintenance and record keeping ...,"458,788 followers",Financial Services
636,Bajaj Finserv,Assistant Manager - Fixed Deposits(JR00094858),"Hyderabad, Telangana, India",Nil,Full-time,"10,001+ employees · Financial Services",https://www.linkedin.com/company/bajaj-finserv...,Job Reference\n\nJR00094858\n\nJob Summary\n\n...,"904,082 followers",Financial Services
637,DBS Bank,CASA SALES OFFICER,"Salem, Tamil Nadu, India",Nil,Full-time,"10,001+ employees · Banking",https://www.linkedin.com/company/dbs-bank/life/,Key Accountability: The key accountability of ...,"790,457 followers",Banking


Saving Final DF:

In [71]:
new_df.to_csv('New_uncleaned_data.csv')